In [ ]:
PATH = 'drive/MyDrive/datasets/Refind_KDF@SIGIR2023_data/public_dat/'

In [ ]:
import pandas as pd

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Embedding Creation

In [ ]:
!pip install -q transformers sentence_transformers

In [ ]:
from transformers import AutoTokenizer
import torch
from transformers import BertTokenizer
from transformers import BertModel

In [ ]:
from collections import Counter

In [ ]:
from torch.utils.data import DataLoader
import math
# from sentence_transformers import LoggingHandler, util
# from sentence_transformers.cross_encoder import CrossEncoder
# from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
# from sentence_transformers.readers import InputExample
# import logging
from datetime import datetime
import os
import gzip
import csv

In [ ]:
train = pd.read_excel(PATH + 'train.xlsx')
train.head()

,id,docid,relation,rel_group,token,e1_start,e1_end,e2_start,e2_end,e1_type,e2_type,spacy_pos,spacy_ner,spacy_head,spacy_deprel,sdp,sdp_tok_idx,token_test,e1,e2
0,NaN,2016/2017,no_relation,ORG-ORG,"['warrants', 'to', 'purchase', 'Lumos', 'Netwo...",3,6,9,12,ORG,ORG,"['NNS', 'TO', 'VB', 'NNP', 'NNP', 'NNP', 'JJ',...","['O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'O',...","[0, 2, 0, 5, 5, 7, 7, 2, 7, 11, 11, 7, 0, 0]","['ROOT', 'aux', 'acl', 'nmod', 'nmod', 'nmod',...","['Lumos Networks Corp.', 'stock', 'purchase', ...","[7, 2]",warrantstopurchaselumosnetworkscorpcommonstock...,lumosnetworkscorp,thepamplonaentities
1,NaN,2016/2017,no_relation,ORG-ORG,"['warrants', 'to', 'purchase', 'Lumos', 'Netwo...",9,12,3,6,ORG,ORG,"['NNS', 'TO', 'VB', 'NNP', 'NNP', 'NNP', 'JJ',...","['O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'O', 'O',...","[0, 2, 0, 5, 5, 7, 7, 2, 7, 11, 11, 7, 0, 0]","['ROOT', 'aux', 'acl', 'nmod', 'nmod', 'nmod',...","['Lumos Networks Corp.', 'stock', 'purchase', ...","[7, 2]",warrantstopurchaselumosnetworkscorpcommonstock...,thepamplonaentities,lumosnetworkscorp
2,BERTPretrain_10KReports/2017/QTR2/20170518_10-...,2016/2017,no_relation,ORG-ORG,"['turn', 'over', 'to', 'Global', 'Gold', 'at',...",115,117,95,96,ORG,ORG,"['NN', 'RP', 'IN', 'NNP', 'NNP', 'IN', 'PRP$',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[0, 0, 0, 4, 2, 4, 7, 5, 7, 8, 9, 12, 9, 0, 19...","['ROOT', 'prt', 'prep', 'compound', 'pobj', 'p...","['Stockhouse', 'Vartanian', 'pay', 'Global Gold']","[70, 114]",turnovertoglobalgoldatitsofficesinryenewyorkco...,globalgold,stockhouse
3,NaN,2016/2017,no_relation,ORG-ORG,"['ts', 'Eighteen', 'of', 'FelCor', 'Lodging', ...",21,24,15,17,ORG,ORG,"['NNP', 'CD', 'IN', 'NNP', 'NNP', 'NNP', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[8, 8, 1, 7, 7, 7, 7, 2, 8, 8, 13, 10, 10, 9, ...","['dep', 'nsubj', 'prep', 'compound', 'compound...","['Embassy Suites', 'agreements', 'separate', '...","[13, 19, 25]",tseighteenoffelcorlodginglpconsolidatedhotelso...,felcorlodginglp,embassysuites
4,NaN,2016/2017,no_relation,ORG-ORG,"['the', 'WFOE', 'will', 'waive', 'and', 'relea...",21,29,1,2,ORG,ORG,"['DT', 'NNP', 'MD', 'VB', 'CC', 'VB', 'PRP', '...","['O', 'ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O'...","[1, 3, 3, 3, 3, 3, 5, 5, 5, 10, 8, 10, 11, 12,...","['det', 'nsubj', 'aux', 'ROOT', 'cc', 'conj', ...","['WFOE', 'waive', 'unconditionally', 'claims',...","[3, 7, 10, 11, 15, 31, 19]",thewfoewillwaiveandreleaseyouunconditionallyfr...,youondemandholdingsincs,wfoe


In [ ]:
train['token'] = train['token'].apply(lambda x : eval(x))
train['full_text'] = train['token'].apply(lambda x : ' '.join(x))
train['entity_1'] = train.apply(lambda x : ' '.join(x.token[int(x.e1_start) : int(x.e1_end)]), axis=1)
train['entity_2'] = train.apply(lambda x : ' '.join(x.token[int(x.e2_start) : int(x.e2_end)]), axis=1)

In [ ]:
train['full_text_entity_1_masked'] =  train.apply(lambda x : ' '.join(x.token[0 : int(x.e1_start)] + ['[MASK]'] + x.token[int(x.e1_end):]), axis=1)
train['full_text_entity_2_masked'] =  train.apply(lambda x : ' '.join(x.token[0 : int(x.e2_start)] + ['[MASK]'] + x.token[int(x.e2_end):]), axis=1)


In [ ]:
train_use = train[['full_text', 'entity_1', 'entity_2', 'e1_type',	'e2_type', 'full_text_entity_1_masked', 'full_text_entity_2_masked', 'rel_group', 'relation']]
train_use

,full_text,entity_1,entity_2,e1_type,e2_type,full_text_entity_1_masked,full_text_entity_2_masked,rel_group,relation
0,warrants to purchase Lumos Networks Corp. comm...,Lumos Networks Corp.,the Pamplona Entities,ORG,ORG,"warrants to purchase [MASK] common stock , the...",warrants to purchase Lumos Networks Corp. comm...,ORG-ORG,no_relation
1,warrants to purchase Lumos Networks Corp. comm...,the Pamplona Entities,Lumos Networks Corp.,ORG,ORG,warrants to purchase Lumos Networks Corp. comm...,"warrants to purchase [MASK] common stock , the...",ORG-ORG,no_relation
2,turn over to Global Gold at its offices in Rye...,Global Gold,Stockhouse,ORG,ORG,turn over to Global Gold at its offices in Rye...,turn over to Global Gold at its offices in Rye...,ORG-ORG,no_relation
3,ts Eighteen of FelCor Lodging LP Consolidated ...,FelCor Lodging LP,Embassy Suites,ORG,ORG,ts Eighteen of FelCor Lodging LP Consolidated ...,ts Eighteen of FelCor Lodging LP Consolidated ...,ORG-ORG,no_relation
4,the WFOE will waive and release you unconditio...,"YOU ON DEMAND HOLDINGS , INC . s",WFOE,ORG,ORG,the WFOE will waive and release you unconditio...,the [MASK] will waive and release you uncondit...,ORG-ORG,no_relation
...,...,...,...,...,...,...,...,...,...
20065,"On April 15 , 2010 , Demand Pooling , Inc. ent...","Demand Pooling , Inc.","Demand Pooling Global Services , LLC",ORG,ORG,"On April 15 , 2010 , [MASK] entered into a Lic...","On April 15 , 2010 , Demand Pooling , Inc. ent...",ORG-ORG,org:org:agreement_with
20066,"On July 17 , 2017 , Greater Cannabis Company ,...","Greater Cannabis Company , Inc.","Xeraflop Technologies , Inc.",ORG,ORG,"On July 17 , 2017 , [MASK] entered into a Conv...","On July 17 , 2017 , Greater Cannabis Company ,...",ORG-ORG,org:org:agreement_with
20067,"In November 2016 , Xcede entered into another ...",Xcede,Cook,ORG,ORG,"In November 2016 , [MASK] entered into another...","In November 2016 , Xcede entered into another ...",ORG-ORG,org:org:agreement_with
20068,"On August 5 , 2015 , Achaogen Inc entered into...",Achaogen Inc,Solar Capital Ltd,ORG,ORG,"On August 5 , 2015 , [MASK] entered into a loa...","On August 5 , 2015 , Achaogen Inc entered into...",ORG-ORG,org:org:agreement_with


In [ ]:
train_use['relation'].value_counts()

no_relation                 9128
pers:title:title            3126
org:gpe:operations_in       2832
pers:org:employee_of        1733
org:org:agreement_with       653
org:date:formed_on           448
pers:org:member_of           441
org:org:subsidiary_of        386
org:org:shares_of            286
org:money:revenue_of         217
org:money:loss_of            141
org:gpe:headquartered_in     135
org:date:acquired_on         134
pers:org:founder_of           92
org:gpe:formed_in             81
org:org:acquired_by           55
pers:univ:employee_of         53
pers:gov_agy:member_of        40
pers:univ:attended            30
pers:univ:member_of           23
org:money:profit_of           20
org:money:cost_of             16
Name: relation, dtype: int64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base")
model = BertModel.from_pretrained("nlpaueb/sec-bert-base")

Some weights of the model checkpoint at nlpaueb/sec-bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_embeddings(processed_text):
  tokenized_text = tokenizer.tokenize(processed_text, max_length = 512)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  try:
    index = tokenized_text.index('[MASK]')
    tokens_tensor = torch.tensor([indexed_tokens])
  except ValueError:
    tokenized_text = tokenizer.tokenize(processed_text)
    tokenized_text_use = tokenized_text[max(0, tokenized_text.index('[MASK]')-255):min(tokenized_text.index('[MASK]')+255, len(tokenized_text))]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text_use)
    index = tokenized_text_use.index('[MASK]')
    tokens_tensor = torch.tensor([indexed_tokens])


  model.eval()
  with torch.no_grad():
    last_hidden_states = model(tokens_tensor)[0]

  embedding_of_num = last_hidden_states[:,index,:]
  embedding_of_num_use = embedding_of_num[0].cpu().detach().numpy()
  return embedding_of_num_use

In [ ]:
# As this step involves heavy compuattion, it was done in three steps
# instances from 0 to 2840
# instances from 2840 to 16625
# instances from 16625 till end

all_embedding1 = []
all_embedding2 = []
N = 16626 #2840
for preprocessed_text1, preprocessed_text2 in zip(train_use['full_text_entity_1_masked'].tolist()[N:], train_use['full_text_entity_2_masked'].tolist()[N:]):
  embedding1 = get_embeddings(preprocessed_text1)
  embedding2 = get_embeddings(preprocessed_text2)
  all_embedding1.append(embedding1)
  all_embedding2.append(embedding2)

In [ ]:
# pd.DataFrame(all_embedding1).to_csv(PATH + "train_entity_1_embedding_0_2840.csv", index = False)
# pd.DataFrame(all_embedding2).to_csv(PATH + "train_entity_2_embedding_0_2840.csv", index = False)


In [ ]:
# pd.DataFrame(all_embedding1).to_csv(PATH + "train_entity_1_embedding_2840_16625.csv", index = False)
# pd.DataFrame(all_embedding2).to_csv(PATH + "train_entity_2_embedding_2840_16625.csv", index = False)


In [ ]:
pd.DataFrame(all_embedding1).to_csv(PATH + "train_entity_1_embedding_16626_end.csv", index = False)
pd.DataFrame(all_embedding2).to_csv(PATH + "train_entity_2_embedding_16626_end.csv", index = False)


In [ ]:
dev = pd.read_excel(PATH + 'dev.xlsx')

dev['token'] = dev['token'].apply(lambda x : eval(x))
dev['full_text'] = dev['token'].apply(lambda x : ' '.join(x))
dev['entity_1'] = dev.apply(lambda x : ' '.join(x.token[int(x.e1_start) : int(x.e1_end)]), axis=1)
dev['entity_2'] = dev.apply(lambda x : ' '.join(x.token[int(x.e2_start) : int(x.e2_end)]), axis=1)

dev['full_text_entity_1_masked'] =  dev.apply(lambda x : ' '.join(x.token[0 : int(x.e1_start)] + ['[MASK]'] + x.token[int(x.e1_end):]), axis=1)
dev['full_text_entity_2_masked'] =  dev.apply(lambda x : ' '.join(x.token[0 : int(x.e2_start)] + ['[MASK]'] + x.token[int(x.e2_end):]), axis=1)

dev_use = dev[['full_text', 'entity_1', 'entity_2', 'e1_type',	'e2_type', 'full_text_entity_1_masked', 'full_text_entity_2_masked', 'rel_group', 'relation']]


all_embedding1 = []
all_embedding2 = []
N = 0
for preprocessed_text1, preprocessed_text2 in zip(dev_use['full_text_entity_1_masked'].tolist()[N:], dev_use['full_text_entity_2_masked'].tolist()[N:]):
  embedding1 = get_embeddings(preprocessed_text1)
  embedding2 = get_embeddings(preprocessed_text2)
  all_embedding1.append(embedding1)
  all_embedding2.append(embedding2)

pd.DataFrame(all_embedding1).to_csv(PATH + "dev_entity_1_embedding_all.csv", index = False)
pd.DataFrame(all_embedding2).to_csv(PATH + "dev_entity_2_embedding_all.csv", index = False)

In [ ]:
test = pd.read_excel(PATH + 'test.xlsx')

test['token'] = test['token'].apply(lambda x : eval(x))
test['full_text'] = test['token'].apply(lambda x : ' '.join(x))
test['entity_1'] = test.apply(lambda x : ' '.join(x.token[int(x.e1_start) : int(x.e1_end)]), axis=1)
test['entity_2'] = test.apply(lambda x : ' '.join(x.token[int(x.e2_start) : int(x.e2_end)]), axis=1)

test['full_text_entity_1_masked'] =  test.apply(lambda x : ' '.join(x.token[0 : int(x.e1_start)] + ['[MASK]'] + x.token[int(x.e1_end):]), axis=1)
test['full_text_entity_2_masked'] =  test.apply(lambda x : ' '.join(x.token[0 : int(x.e2_start)] + ['[MASK]'] + x.token[int(x.e2_end):]), axis=1)

test_use = test[['full_text', 'entity_1', 'entity_2', 'e1_type',	'e2_type', 'full_text_entity_1_masked', 'full_text_entity_2_masked', 'rel_group', 'relation']]


all_embedding1 = []
all_embedding2 = []
N = 0
for preprocessed_text1, preprocessed_text2 in zip(test_use['full_text_entity_1_masked'].tolist()[N:], test_use['full_text_entity_2_masked'].tolist()[N:]):
  embedding1 = get_embeddings(preprocessed_text1)
  embedding2 = get_embeddings(preprocessed_text2)
  all_embedding1.append(embedding1)
  all_embedding2.append(embedding2)

pd.DataFrame(all_embedding1).to_csv(PATH + "test_entity_1_embedding_all.csv", index = False)
pd.DataFrame(all_embedding2).to_csv(PATH + "test_entity_2_embedding_all.csv", index = False)

In [ ]:
data = pd.read_excel(PATH + "train.xlsx")

In [ ]:
data.columns

Index(['id', 'docid', 'relation', 'rel_group', 'token', 'e1_start', 'e1_end',
       'e2_start', 'e2_end', 'e1_type', 'e2_type', 'spacy_pos', 'spacy_ner',
       'spacy_head', 'spacy_deprel', 'sdp', 'sdp_tok_idx', 'token_test', 'e1',
       'e2'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_X = pd.concat([pd.get_dummies(pd.read_excel(PATH + "train.xlsx")[['rel_group', 'e1_type', 'e2_type']]).reset_index(drop = True), pd.read_csv(PATH + "train_entity_1_embedding_0_2839.csv").append(pd.read_csv(PATH + "train_entity_1_embedding_2840_16625.csv")).append(pd.read_csv(PATH + "train_entity_1_embedding_16626_end.csv")).reset_index(drop = True), pd.read_csv(PATH + "train_entity_2_embedding_0_2839.csv").append(pd.read_csv(PATH + "train_entity_2_embedding_2840_16625.csv")).append(pd.read_csv(PATH + "train_entity_2_embedding_16626_end.csv")).reset_index(drop = True)], axis = 1)
train_y = pd.read_excel(PATH + "train.xlsx")['relation']


valid_X = pd.concat([pd.get_dummies(pd.read_excel(PATH + "dev.xlsx")[['rel_group', 'e1_type', 'e2_type']]).reset_index(drop = True), pd.read_csv(PATH + "dev_entity_1_embedding_all.csv"), pd.read_csv(PATH + "dev_entity_2_embedding_all.csv")], axis = 1)
valid_y = pd.read_excel(PATH + "dev.xlsx")['relation']

test_X = pd.concat([pd.get_dummies(pd.read_excel(PATH + "test.xlsx")[['rel_group', 'e1_type', 'e2_type']]).reset_index(drop = True), pd.read_csv(PATH + "test_entity_1_embedding_all.csv"), pd.read_csv(PATH + "test_entity_2_embedding_all.csv")], axis = 1)
test_y = pd.read_excel(PATH + "test.xlsx")['relation']

<ipython-input-10-6ff489112241>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_X = pd.concat([pd.get_dummies(pd.read_excel(PATH + "train.xlsx")[['rel_group', 'e1_type', 'e2_type']]).reset_index(drop = True), pd.read_csv(PATH + "train_entity_1_embedding_0_2839.csv").append(pd.read_csv(PATH + "train_entity_1_embedding_2840_16625.csv")).append(pd.read_csv(PATH + "train_entity_1_embedding_16626_end.csv")).reset_index(drop = True), pd.read_csv(PATH + "train_entity_2_embedding_0_2839.csv").append(pd.read_csv(PATH + "train_entity_2_embedding_2840_16625.csv")).append(pd.read_csv(PATH + "train_entity_2_embedding_16626_end.csv")).reset_index(drop = True)], axis = 1)
<ipython-input-10-6ff489112241>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_X = pd.concat([pd.get_dummies(pd.read_excel(PATH + "train.xlsx

In [ ]:
clf = MLPClassifier(random_state=1, hidden_layer_sizes = [512,128], max_iter=300).fit(train_X, train_y)

In [ ]:
train_y_predicted = clf.predict(train_X)

print("precision", precision_score(train_y, train_y_predicted, average='weighted'))
print("recall", recall_score(train_y, train_y_predicted, average='weighted'))
print("f1", f1_score(train_y, train_y_predicted, average='weighted'))

precision 0.9969158774937321
recall 0.9969108121574489
f1 0.996907178538138


In [ ]:
valid_y_predicted = clf.predict(valid_X)

print("precision", precision_score(valid_y, valid_y_predicted, average='weighted'))
print("recall", recall_score(valid_y, valid_y_predicted, average='weighted'))
print("f1", f1_score(valid_y, valid_y_predicted, average='weighted'))

precision 0.8220207556215636
recall 0.8190896423594983
f1 0.8172376829946659


In [ ]:
test_y_predicted = clf.predict(test_X)

print("precision", precision_score(test_y, test_y_predicted, average='weighted'))
print("recall", recall_score(test_y, test_y_predicted, average='weighted'))
print("f1", f1_score(test_y, test_y_predicted, average='weighted'))

precision 0.7476760212745514
recall 0.7430232558139535
f1 0.7361787255017052


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pickle
filename = PATH + 'REFinD_best_model.pickle'
pickle.dump(clf, open(filename, 'wb'))

## Performance on 500 randomly selected instances from test set

In [ ]:
test_X_y_sample500 = test_X.copy()
test_X_y_sample500['relation'] = test_y
test_X_y_sample500 = test_X_y_sample500.sample(500, random_state=0)

test_X_sample500 = test_X_y_sample500.drop(['relation'], axis = 1)
test_y_predicted_sample500 = clf.predict(test_X_sample500)
test_y_sample500 = test_X_y_sample500['relation']

print("precision", precision_score(test_y_sample500 , test_y_predicted_sample500 , average='weighted'))
print("recall", recall_score(test_y_sample500 , test_y_predicted_sample500 , average='weighted'))
print("f1", f1_score(test_y_sample500, test_y_predicted_sample500, average='weighted'))

precision 0.7259239178311919
recall 0.724
f1 0.7169195686515631


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
Counter(test_y_predicted)

Counter({'no_relation': 2198,
         'org:org:shares_of': 54,
         'org:org:subsidiary_of': 79,
         'org:org:acquired_by': 4,
         'pers:org:employee_of': 528,
         'org:org:agreement_with': 59,
         'pers:univ:attended': 6,
         'pers:title:title': 560,
         'pers:org:founder_of': 14,
         'pers:org:member_of': 88,
         'org:gpe:operations_in': 430,
         'org:gpe:headquartered_in': 28,
         'org:gpe:formed_in': 18,
         'org:money:revenue_of': 42,
         'org:money:loss_of': 48,
         'org:money:profit_of': 10,
         'org:date:formed_on': 94,
         'org:date:acquired_on': 34,
         'pers:univ:employee_of': 2,
         'pers:gov_agy:member_of': 2,
         'org:money:cost_of': 1,
         'pers:univ:member_of': 1})

In [ ]:
test_y_predicted

array(['no_relation', 'no_relation', 'no_relation', ..., 'no_relation',
       'no_relation', 'no_relation'], dtype='<U24')

In [ ]:
predicted_num_dict = {'no_relation':0,
"org:date:formed_on":1,
"org:gpe:operations_in":2,
"pers:org:member_of":3,
"pers:org:employee_of":4,
"pers:gov_agy:member_of":5,
"org:org:acquired_by":6,
"org:money:loss_of":7,
"org:gpe:headquartered_in":8,
"pers:univ:employee_of":9,
"org:date:acquired_on":10,
"pers:univ:attended":11,
"org:gpe:formed_in":12,
"org:money:profit_of":13,
"org:money:cost_of":14,
"org:org:subsidiary_of":15,
"org:org:shares_of":16,
"pers:org:founder_of":17,
"pers:title:title":18,
"org:money:revenue_of":19,
"org:org:agreement_with":20,
"pers:univ:member_of":21}

In [ ]:
pd.DataFrame([predicted_num_dict[i] for i in test_y_predicted]).to_csv("LIPI.csv", index = False, header = None)

In [ ]:
!wc -l LIPI.csv

4300 LIPI.csv


In [ ]:
!zip -r "LIPI_SUBMISSION.zip" "LIPI.csv"

  adding: LIPI.csv (deflated 87%)


In [ ]:
from google.colab import files
files.download('LIPI_SUBMISSION.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>